In [12]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [13]:
cox_df = pd.read_csv('data/cox_inhibitor.tsv', sep='\t')
cox_inhibitors = ['-'.join(x.upper().split(' '))  for x in cox_df['Drug'].values]
hdac_df = pd.read_csv('data/hdac_inhibitors.csv', index_col=0)
hdac_inhibitors = ['-'.join(x.upper().split(' ')) for x in hdac_df['Drug'].values]
cdk_df = pd.read_csv('data/CDK inhibitor.txt', sep='\t')
cdk_inhibitors = ['-'.join(x.upper().split(' ')) for x in cdk_df['Name'].values]

In [ ]:
ranking_dict = {
    'pvalue': {'scores': [], 'labels': []},
    'oddsRatio': {'scores': [], 'labels': []}, 
    'pvalueUp': {'scores': [], 'labels': []}, 
    'pvalueDown': {'scores': [], 'labels': []}, 
    'oddsRatioUp': {'scores': [], 'labels': []},
    'oddsRatioDown': {'scores': [], 'labels': []}
}

for metric in ranking_dict.keys():
    for term in tqdm(os.listdir('data/hdac_out')):
        rank_df = pd.read_csv(f'data/hdac_out/{term}', sep='\t', index_col=0)
        rank_df = rank_df[(rank_df['pvalue'] < 0.01)]
        if 'pvalue' in metric:
            rank_df.sort_values(by=metric, inplace=True, ascending=True)
        else:
            rank_df.sort_values(by=metric, inplace=True, ascending=False)
        rank_df.reset_index(drop=True, inplace=True)
        rank_df['labels'] = [1 if x.upper() in hdac_inhibitors else 0 for x in rank_df['drug']]
        rank_df['scores'] = 1 -  ((rank_df.index.values) / len(rank_df))
        ranking_dict[metric]['scores'].extend(list(rank_df['scores']))
        ranking_dict[metric]['labels'].extend(list(rank_df['labels']))
        

In [ ]:
plt.figure(figsize=(8, 6))
for metric in ranking_dict.keys():   
    fpr, tpr, thresholds = roc_curve(ranking_dict[metric]['labels'], ranking_dict[metric]['scores'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{metric} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random')
plt.savefig('data/figures/adj_pvalue_hdac.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import os
ranking_dict_cdk = {
    'pvalue': {'scores': [], 'labels': []},
    'oddsRatio': {'scores': [], 'labels': []}, 
    'pvalueUp': {'scores': [], 'labels': []}, 
    'pvalueDown': {'scores': [], 'labels': []}, 
    'oddsRatioUp': {'scores': [], 'labels': []},
    'oddsRatioDown': {'scores': [], 'labels': []}
}

for metric in ranking_dict_cdk.keys():
    for term in tqdm(os.listdir('data/cdk_out')):
        rank_df = pd.read_csv(f'data/cdk_out/{term}', sep='\t', index_col=0)
        rank_df = rank_df[(rank_df['pvalue'] < 0.01)]
        if 'pvalue' in metric:
            rank_df.sort_values(by=metric, inplace=True, ascending=True)
        else:
            rank_df.sort_values(by=metric, inplace=True, ascending=False)
        rank_df.reset_index(drop=True, inplace=True)
        rank_df['labels'] = [1 if x.upper() in cdk_inhibitors else 0 for x in rank_df['drug']]
        rank_df['scores'] = 1 -  ((rank_df.index.values) / len(rank_df))
        ranking_dict_cdk[metric]['scores'].extend(list(rank_df['scores']))
        ranking_dict_cdk[metric]['labels'].extend(list(rank_df['labels']))

In [ ]:
plt.figure(figsize=(8, 6))
for metric in ranking_dict_cdk.keys():   
    fpr, tpr, thresholds = roc_curve(ranking_dict_cdk[metric]['labels'], ranking_dict_cdk[metric]['scores'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{metric} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
#plt.xlim([0, 0.2])
#plt.ylim([0, .4])
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random')
plt.show()

In [16]:
def bootstrap_roc_curve(ones: list, zeros: list, n: int):
    """
    Calculate the mean area under the ROC curve (AUC) and the mean true positive rate (TPR)
    using bootstrap resampling.

    Parameters:
    ones (array-like): The positive class labels.
    zeros (array-like): The negative class labels.
    n (int): The number of bootstrap iterations.

    Returns:
    dict: A dictionary containing the mean AUC and the mean TPR.

    """
    base_fpr = np.linspace(0, 1, 50)
    size_group1 = len(ones)
    sum_auc = 0
    tprs = []
    for i in range(n):
        zeros_sampled = np.random.choice(zeros, size=size_group1, replace=True)
        fpr, tpr, _ = roc_curve(np.concatenate([np.ones_like(ones), np.zeros_like(zeros_sampled)]),
                                np.concatenate([ones, zeros_sampled]), drop_intermediate=False)
        roc_auc = auc(fpr, tpr)
        sum_auc += roc_auc
        tpr = np.interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tprs.append(list(tpr))

    auc_mean = sum_auc / n

    tpr_mean = np.mean(np.array(tprs), axis=0)
    return {'auc': auc_mean, 'approx': tpr_mean}


def plot_roc_curve(roc_vals: dict, name: str, n_bootstrap=5000):
    """
    Plots the ROC curve for the given ROC values.

    Parameters:
    roc_vals (dict): Dictionary containing the ROC values for different libraries.
    name (str): name of output figure
    n_bootstrap (int): Number of bootstrap iterations for calculating confidence intervals. Default is 5000.

    Returns:
    None
    """

    base_fpr = np.linspace(0, 1, 50)

    lib_curves = {}
    for lib in tqdm(roc_vals):
        scores = roc_vals[lib]['scores']
        labels = np.array(roc_vals[lib]['labels'])
        idx1 = np.where(labels == 1)
        idx0 = np.where(labels == 0)
        tp = np.array(scores)[idx1]
        fp = np.array(scores)[idx0]
        bootstrapped_res = bootstrap_roc_curve(tp, fp, n_bootstrap)
        print(lib, bootstrapped_res['auc'])
        lib_curves[lib] = bootstrapped_res

    # Plotting
    fig = plt.figure(figsize=(8, 6))
    for i, lib in enumerate(list(lib_curves)):
        plt.plot(base_fpr, lib_curves[lib]['approx'], label=f"{lib} AUC: {np.round(lib_curves[lib]['auc'], 3)}")

    plt.plot([0, 1], [0, 1], color='black', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

In [ ]:
plot_roc_curve(ranking_dict, 'test')

In [ ]:
crispr_hdac_ranks = {
    'pvalue': {'scores': [], 'labels': []},
    'oddsRatio': {'scores': [], 'labels': []},
    'pvalueUp': {'scores': [], 'labels': []}, 
    'pvalueDown': {'scores': [], 'labels': []}, 
    'oddsRatioUp': {'scores': [], 'labels': []},
    'oddsRatioDown': {'scores': [], 'labels': []}
}

for metric in crispr_hdac_ranks.keys():
    for term in tqdm(os.listdir('data/hdac_out')):
        rank_df = pd.read_csv(f'data/hdac_out/{term}', sep='\t', index_col=0)   
        rank_df = rank_df[rank_df['drug'].str.contains(' ')]
        rank_df = rank_df[(rank_df['pvalue'] < 0.05)]
        if 'pvalue' in metric:
            rank_df.sort_values(by=metric, inplace=True, ascending=True)
        else:
            rank_df.sort_values(by=metric, inplace=True, ascending=False)
        rank_df.reset_index(drop=True, inplace=True)
        rank_df['labels'] = [1 if 'HDAC' in x.upper() else 0 for x in rank_df['drug']]
        rank_df['scores'] = 1 -  ((rank_df.index.values) / len(rank_df))
        crispr_hdac_ranks[metric]['scores'].extend(list(rank_df['scores']))
        crispr_hdac_ranks[metric]['labels'].extend(list(rank_df['labels']))

In [ ]:
score_genes = {}

for term in tqdm(os.listdir('data/hdac_out')):
    rank_df = pd.read_csv(f'data/hdac_out/{term}', sep='\t', index_col=0)
    rank_df = rank_df[rank_df['drug'].str.contains(' ')]
    rank_df.reset_index(drop=True, inplace=True)
    rank_df['score'] = 1 -  ((rank_df.index.values) / len(rank_df))
    rank_df = rank_df[(rank_df['pvalue'] < 0.05)]
    for pert in rank_df['drug'].unique():
        if pert not in score_genes:
            score_genes[pert] = []
        pert_df = rank_df[rank_df['drug'] == pert]
        score_genes[pert].extend(list(pert_df['score'].values))
        


In [ ]:
gene_data = []
for g in score_genes:
    gene_data.append([g, np.mean(score_genes[g]), np.std(score_genes[g]), len(score_genes[g])])
gene_hdac_df = pd.DataFrame(gene_data, columns=['gene', 'mean', 'std', 'num appearances']).sort_values(by='mean', ascending=False)
gene_hdac_df.set_index('gene', inplace=True)
n = len(os.listdir('data/hdac_out'))
gene_hdac_df['percent appearances'] = ((gene_hdac_df['num appearances'] / n) * 100).round(2)
display(gene_hdac_df[ gene_hdac_df.index.str.contains('HDAC')].sort_values(by='num appearances', ascending=False).head(10))

In [ ]:
score_genes_cdk = {}

for term in tqdm(os.listdir('data/cdk_out')):
    rank_df = pd.read_csv(f'data/cdk_out/{term}', sep='\t', index_col=0)
    rank_df = rank_df[rank_df['drug'].str.contains(' ')]
    rank_df.reset_index(drop=True, inplace=True)
    rank_df['score'] = 1 -  ((rank_df.index.values) / len(rank_df))
    rank_df = rank_df[(rank_df['pvalue'] < 0.05)]
    for pert in rank_df['drug'].unique():
        if pert not in score_genes_cdk:
            score_genes_cdk[pert] = []
        pert_df = rank_df[rank_df['drug'] == pert]
        score_genes_cdk[pert].extend(list(pert_df['score'].values))

In [ ]:
gene_data = []
for g in score_genes_cdk:
    gene_data.append([g, np.mean(score_genes_cdk[g]), np.std(score_genes_cdk[g]), len(score_genes_cdk[g])])
gene_cdk_df = pd.DataFrame(gene_data, columns=['gene', 'mean scaled rank', 'std', 'num appearances']).sort_values(by='mean scaled rank', ascending=False)
gene_cdk_df.set_index('gene', inplace=True)
n = len(os.listdir('data/cdk_out'))
gene_cdk_df['percent appearances'] = ((gene_cdk_df['num appearances'] / n) * 100).round(2)
display(gene_cdk_df[ gene_cdk_df.index.str.contains('CDK')].sort_values(by='num appearances', ascending=False).head(10))

In [ ]:
gene_cdk_df.sort_values(by='num appearances', ascending=False).head(10)

In [ ]:
#gene_hdac_df[ (gene_hdac_df['num appearances'] > 10) & (gene_hdac_df['mean'] > 0.9)]
#gene_hdac_df[ (gene_hdac_df['num appearances'] > 20) & (gene_hdac_df['mean'] > 0.95)]
top_hdac = set(gene_hdac_df.sort_values(by='num appearances', ascending=False).index.values[:250])
top_cdk  = set(gene_cdk_df.sort_values(by='num appearances', ascending=False).index.values[:250])
diff_hdac = top_hdac.difference(top_cdk)
diff_cdk = top_cdk.difference(top_hdac)
intersection = top_hdac.intersection(top_cdk)

import matplotlib_venn
matplotlib_venn.venn2([top_hdac, top_cdk], set_labels=('HDAC top 250 KOs', 'CDK top 250 KOs'))
plt.savefig('venn_hdac_cdk.png', dpi=300)

In [ ]:
cdk_unique = [g for g in top_cdk if g not in top_hdac]
hdac_unique = [g for g in top_hdac if g not in top_cdk]
len(cdk_unique), len(hdac_unique), len(intersection)

In [64]:
with open('hdac_unique.txt', 'w') as f:
    f.write('\n'.join(hdac_unique))

with open('cdk_unique.txt', 'w') as f:
    f.write('\n'.join(cdk_unique))

with open('intersection.txt', 'w') as f:
    f.write('\n'.join(intersection))

In [ ]:
plt.figure(figsize=(8, 6))
for metric in crispr_hdac_ranks.keys():   
    fpr, tpr, thresholds = roc_curve(crispr_hdac_ranks[metric]['labels'], crispr_hdac_ranks[metric]['scores'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{metric} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random')
plt.show()

In [ ]:
crispr_cdk_ranks = {
    'pvalue': {'scores': [], 'labels': []},
    'oddsRatio': {'scores': [], 'labels': []},
    'pvalueUp': {'scores': [], 'labels': []}, 
    'pvalueDown': {'scores': [], 'labels': []}, 
    'oddsRatioUp': {'scores': [], 'labels': []},
    'oddsRatioDown': {'scores': [], 'labels': []}
}

for metric in crispr_cdk_ranks.keys():
    for term in tqdm(os.listdir('data/cdk_out')):
        rank_df = pd.read_csv(f'data/cdk_out/{term}', sep='\t', index_col=0)   
        rank_df = rank_df[rank_df['drug'].str.contains(' ')]
        rank_df = rank_df[(rank_df['pvalue'] < 0.05)]
        if 'pvalue' in metric:
            rank_df.sort_values(by=metric, inplace=True, ascending=True)
        else:
            rank_df.sort_values(by=metric, inplace=True, ascending=False)
        rank_df.reset_index(drop=True, inplace=True)
        rank_df['labels'] = [1 if 'CDK' in x.upper() else 0 for x in rank_df['drug']]
        rank_df['scores'] = 1 -  ((rank_df.index.values) / len(rank_df))
        crispr_cdk_ranks[metric]['scores'].extend(list(rank_df['scores']))
        crispr_cdk_ranks[metric]['labels'].extend(list(rank_df['labels']))

In [ ]:
plt.figure(figsize=(8, 6))
for metric in crispr_cdk_ranks.keys():   
    fpr, tpr, thresholds = roc_curve(crispr_cdk_ranks[metric]['labels'], crispr_cdk_ranks[metric]['scores'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{metric} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
plt.ylim([0, .4])
plt.xlim([0, 0.2])
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random')
plt.show()

In [29]:
hdac_term_search = pd.read_csv('data/results (10).tsv', sep='\t')
cdk_term_search = pd.read_csv('data/results (9).tsv', sep='\t')

In [ ]:
hdac_term_search['KO'] = hdac_term_search['term'].map(lambda x: x.split('_')[4])
hdac_term_search = hdac_term_search[hdac_term_search['KO'].str.contains(' ')]
hdac_term_search['KO'] = hdac_term_search['KO'].map(lambda x: x.split(' ')[0])
hdac_kos = hdac_term_search['KO'].values
set(hdac_kos)

In [31]:
cdk_term_search['KO'] = cdk_term_search['term'].map(lambda x: x.split('_')[4])
cdk_term_search = cdk_term_search[cdk_term_search['KO'].str.contains(' ')]
cdk_term_search['KO'] = cdk_term_search['KO'].map(lambda x: x.split(' ')[0])
cdk_kos = cdk_term_search['KO'].values